<a href="https://colab.research.google.com/github/yukinaga/llm_mechanism/blob/main/section_3/01_simple_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなBERTの実装
訓練済みのBERTのモデルを使用し、文章の一部の予測、及び2つの文章が連続しているかどうかの判定を行います。

## ライブラリのインストール
ライブラリTransformers、および必要なライブラリのインストールを行います。

In [ ]:
!pip install transformers==4.26.0

## 文章の一部の予測
文章における一部の単語をMASKし、それをBERTのモデルを使って予測します。

In [ ]:
import torch
from transformers import BertForMaskedLM
from transformers import BertTokenizer

text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)

文章の一部をMASKします。

In [ ]:
msk_idx = 3
words[msk_idx] = "[MASK]"  # 単語を[MASK]に置き換える
print(words)

単語を対応するインデックスに変換します。

In [ ]:
word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids])  # テンソルに変換
print(word_tensor)

BERTのモデルを使って予測を行います。

In [ ]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
msk_model.eval()

x = word_tensor
y = msk_model(x)  # 予測
result = y[0]
print(result.size())  # 結果の形状

_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 最も大きい5つの値
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # インデックスを単語に変換
print(result_words)

## 文章が連続しているかどうかの判定
BERTのモデルを使って、2つの文章が連続しているかどうかの判定を行います。  
以下の関数`show_continuity`では、2つの文章の連続性を判定し、表示します。

In [ ]:
from transformers import BertForNextSentencePrediction

nsp_model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")
nsp_model.eval()  # 評価モード

def show_continuity(text1, text2):
    # トークナイズ
    tokenized = tokenizer(text1, text2, return_tensors="pt")
    print("Tokenized:", tokenized)

    # 予測と結果の表示
    y = nsp_model(**tokenized)  # 予測
    print("Result:", y)
    pred = torch.softmax(y.logits, dim=1)  # Softmax関数で確率に変換
    print(str(pred[0][0].item()*100) + "%の確率で連続しています。")

`show_continuity`関数に、自然につながる2つの文章を与えます。

In [ ]:
text1 = "What is baseball ?"
text2 = "It is a game of hitting the ball with the bat."
show_continuity(text1, text2)

`show_continuity`関数に、自然につながらない2つの文章を与えます。

In [ ]:
text1 = "What is baseball ?"
text2 = "This food is made with flour and milk."
show_continuity(text1, text2)